In [2]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os

In [3]:
# Combine all alphas
def get_stat():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            # print(fileName)
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            # print(df)
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res


In [4]:
stats = get_stat()

In [5]:
stats.describe()

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
count,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000
mean,0.098041,-0.103613,1.505943,-0.828360,0.006605,0.103557,0.368498,0.091564
std,0.088097,0.031894,1.164628,0.283669,0.006607,0.072072,0.065035,0.099181
min,-0.077258,-0.203672,-0.874406,-1.549496,-0.004113,0.020960,0.258523,0.008276
25%,0.033585,-0.120391,0.609141,-1.035955,0.001423,0.056674,0.312163,0.022888
50%,0.086783,-0.104455,1.462913,-0.835905,0.005208,0.068463,0.366992,0.061630
75%,0.159912,-0.089319,2.508969,-0.700563,0.010778,0.132314,0.412113,0.132244
max,0.285464,-0.038737,4.121040,-0.285176,0.021537,0.350836,0.589292,0.748058


In [6]:
good_alphas = list(stats.loc[stats.IR > 2].index)

In [7]:
# combine daily pnl
def get_pnl():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            df = pd.read_csv(file)
            # df = df.set_index('time')
            # df = df['pnl']
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res


In [8]:
pnls = get_pnl()

In [9]:
corr = pnls[good_alphas].corr()

In [10]:
from itertools import combinations
import random
res = []
num = 4
good_candidates = list(
    ((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
random.shuffle(good_candidates)
for cols in combinations(good_candidates, num):
    corr_small = corr.loc[cols, cols]
    if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
        res.append(corr_small)

In [11]:
max_IR = float('-inf')
max_idx = -1
for i in range(len(res)):
    total_IR = np.sum([(stats.loc[stats.index==name].IR) for name in res[i].index])
    if total_IR > max_IR:
        max_IR = total_IR
        max_idx = i

In [12]:
res[max_idx]

,alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoReV008-20days-1neighbors-zz9999;20180101-20201231-Rank-Industry-holding10days,alpha.YaoReV011-20days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days
alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days,1.000000,0.589686,0.046094,0.292743
alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days,0.589686,1.000000,0.387398,0.141018
alpha.YaoReV008-20days-1neighbors-zz9999;20180101-20201231-Rank-Industry-holding10days,0.046094,0.387398,1.000000,0.598635
alpha.YaoReV011-20days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days,0.292743,0.141018,0.598635,1.000000


In [13]:
total_IR

9.30070202333047